# create videos transcripts summaries dataset

In [1]:
import os
os.chdir("..")

In [2]:
ls

app/              data/             notebooks/        requirements.txt


In [3]:
from app.storage.data_lake_pandas import DataLakePandas
from app.llm.llm import SummaryLlm
from app.preprocessing.summary import SummaryProccesing

import pandas as pd

from huggingface_hub import login
from datasets import Dataset,load_dataset, DatasetDict

from dotenv import load_dotenv
load_dotenv()


/Users/andrestrepo/Documents/repos_personal/poc-summary/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

## cargar datos procesados de zona curated del datalake

In [ ]:
dl_pandas = DataLakePandas()

years = [0,2014,2019,2020,2021,2022,2023,2024,2025]

for year in years:
    df_curated = dl_pandas.read_dataframe_from_parquet(
        container_name='curated',
        file_name=f"youtube_data/youtube_data_{year}",
        filter=None,
    )
    df_curated['month'] = pd.to_datetime(df_curated['publish_date']).dt.month
    df_curated['year'] = year
    if year == years[0]:
        df_all = df_curated
    else:
        df_all = pd.concat([df_all, df_curated], ignore_index=True)

In [ ]:
df_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2290 entries, 0 to 2294
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   channel_name      2290 non-null   object        
 1   video_id          2290 non-null   object        
 2   source            2290 non-null   object        
 3   publish_date      1675 non-null   datetime64[ns]
 4   duration          2290 non-null   float64       
 5   last_update_date  2290 non-null   object        
 6   title             2290 non-null   object        
 7   text              2290 non-null   object        
 8   year              2290 non-null   int64         
 9   month             2290 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 196.8+ KB


In [49]:
print(f"fechas de publicación minima y maxima {df_all["publish_date"].min(),df_all["publish_date"].max()}")


fechas de publicación minima y maxima (Timestamp('2014-09-03 00:00:00'), Timestamp('2025-04-17 00:00:00'))


# resumir documentos y analisis de tokens

In [34]:
summary_processing = SummaryProccesing()

In [50]:
new_dataset = summary_processing.add_column_count_tokens(df_all)
new_dataset.head()

,channel_name,video_id,source,publish_date,duration,last_update_date,title,text,year,month,number_of_tokenks
0,Bolsas hoy | Invierte y Crece,MEFcIWdwoEI,https://www.youtube.com/watch?v=MEFcIWdwoEI,NaT,822.0,2025-03-10,sp500 y oro alcanzan sus objetivos proyectados...,sp500 y oro alcanzan sus objetivos proyectados...,0,01,2389
1,Bolsas hoy | Invierte y Crece,yQdTS2296V8,https://www.youtube.com/watch?v=yQdTS2296V8,NaT,1251.0,2025-03-10,la fed te hace perder dinero porque no puede ...,la fed te hace perder dinero porque no puede p...,0,01,3639
2,Bolsas hoy | Invierte y Crece,5uDK19-gLkw,https://www.youtube.com/watch?v=5uDK19-gLkw,NaT,907.0,2025-03-10,máximos históricos del sp 500 son una cuestión...,máximos históricos del sp 500 son una cuestión...,0,01,2847
3,Bolsas hoy | Invierte y Crece,dm-GKckZQG4,https://www.youtube.com/watch?v=dm-GKckZQG4,NaT,1142.0,2025-03-10,mejor aprender a saber qué hacer en vez de bus...,mejor aprender a saber qué hacer en vez de bus...,0,01,3380
4,Bolsas hoy | Invierte y Crece,n4ZxNi2TjaY,https://www.youtube.com/watch?v=n4ZxNi2TjaY,NaT,1055.0,2025-03-10,nuevas proyecciones ya que los planes se han e...,nuevas proyecciones ya que los planes se han e...,0,01,3221


## Percentil 95 del numero de tokens en transcripciones videos por canal

In [52]:
percentiles_95 = new_dataset.groupby("channel_name")["number_of_tokenks"].quantile(0.95)
percentiles_95


channel_name
ARENA ALFA                         5697.60
Bitcoin hoy                        5793.00
Bolsas hoy | Esteban Pérez         5434.50
Bolsas hoy | Invierte y Crece      9193.45
Esteban Perez                      4774.70
USACRYPTONOTICIAS                 22404.00
Name: number_of_tokenks, dtype: float64

## Valores maximos y minimos de tokens en transcripciones videos por canal

In [53]:
## Percentil 95 del numero de tokens por canal
min_max_tokens = new_dataset.groupby("channel_name")["number_of_tokenks"].agg(["min", "max"])
min_max_tokens

,min,max
channel_name,,
ARENA ALFA,162,12280
Bitcoin hoy,1073,6012
Bolsas hoy | Esteban Pérez,687,5835
Bolsas hoy | Invierte y Crece,193,13594
Esteban Perez,156,11955
USACRYPTONOTICIAS,34,42638


## resumir documentos por canal

In [54]:
videos_por_canal = new_dataset.groupby("channel_name")["video_id"].count()
videos_por_canal

channel_name
ARENA ALFA                        145
Bitcoin hoy                        66
Bolsas hoy | Esteban Pérez         26
Bolsas hoy | Invierte y Crece     290
Esteban Perez                     812
USACRYPTONOTICIAS                 951
Name: video_id, dtype: int64

In [4]:
config = {
    "type": "openai",
    "model":'gpt-4o-2024-08-06',
    "api_key": os.getenv("OPENAI_API_KEY"),
    "temperature": 0,
    "max_completion_tokens": 1000,
}

In [5]:
sumary_model = SummaryLlm(config)
sumary_model.model

'gpt-4o-2024-08-06'

In [7]:
print(sumary_model.summary_prompt_template)

Eres un experto de trading y analisis de mercado.
Tu trabajo consiste en elaborar un informe con los topicos mas importantes del siguiente texto:
------------
{context}
------------
Comience el resumen final con una "Introducción" que ofrezca una visión general del tema, seguido
por los puntos más importantes ("Bullet Points"). Termina el resumen con una conclusión.
**Además, extrae los activos mencionados para invertir y crea una lista separada de los mismos.**
Respuesta:


In [ ]:
# Obtener los nombres únicos de los canales
channel_names = new_dataset["channel_name"].unique()

# Crear un diccionario para almacenar los resúmenes por canal
summaries_by_channel = []

In [ ]:
# Generar resumen para cada canal
for channel_name in channel_names:
    print(f"Generando resumen para el canal: {channel_name}")
    df_channel = new_dataset[new_dataset["channel_name"] == channel_name]
    summaries_df = summary_processing.summarize_dataframe(
        df_channel, llm=sumary_model, sleep_time=1
    )
    summaries_df.to_csv(
        f"data/summaries/{channel_name}_summary.csv", sep=";", index=False
    )
    summaries_by_channel.append(summaries_df)

Generando resumen para el canal: USACRYPTONOTICIAS


Summarizing texts: 100%|██████████| 1043/1043 [3:05:42<00:00, 10.68s/it] 


Generando resumen para el canal: Esteban Perez 


Summarizing texts: 100%|██████████| 812/812 [2:00:00<00:00,  8.87s/it]  


Generando resumen para el canal: ARENA ALFA 


Summarizing texts: 100%|██████████| 150/150 [20:50<00:00,  8.34s/it]


Generando resumen para el canal: Bitcoin hoy


Summarizing texts: 100%|██████████| 132/132 [18:34<00:00,  8.44s/it]


Generando resumen para el canal: Bolsas hoy | Esteban Pérez


Summarizing texts: 100%|██████████| 52/52 [06:33<00:00,  7.56s/it]


In [55]:
all_summaries_df = pd.concat(summaries_by_channel, ignore_index=True)

In [62]:
all_summaries_df.drop_duplicates(subset=["video_id"],inplace=True)

In [64]:
all_summaries_df.reset_index(drop=True, inplace=True)
all_summaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   channel_name       2225 non-null   object        
 1   video_id           2225 non-null   object        
 2   source             2225 non-null   object        
 3   publish_date       1610 non-null   datetime64[ns]
 4   duration           2225 non-null   float64       
 5   last_update_date   2225 non-null   object        
 6   title              2225 non-null   object        
 7   text               2225 non-null   object        
 8   year               2225 non-null   int64         
 9   month              2225 non-null   object        
 10  number_of_tokenks  2225 non-null   int64         
 11  prompt             2225 non-null   object        
 12  summary            2225 non-null   object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(9)
memory us

In [ ]:
all_summaries_df = summary_processing.extract_key_terms_from_df(all_summaries_df, column_name='summary')

In [71]:
all_summaries_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2225 entries, 0 to 2224
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   channel_name       2225 non-null   object        
 1   video_id           2225 non-null   object        
 2   source             2225 non-null   object        
 3   publish_date       1610 non-null   datetime64[ns]
 4   duration           2225 non-null   float64       
 5   last_update_date   2225 non-null   object        
 6   title              2225 non-null   object        
 7   text               2225 non-null   object        
 8   year               2225 non-null   int64         
 9   month              2225 non-null   object        
 10  number_of_tokenks  2225 non-null   int64         
 11  prompt             2225 non-null   object        
 12  summary            2225 non-null   object        
 13  key_terms          2212 non-null   object        
dtypes: datet

In [76]:
all_summaries_df[["channel_name","key_terms"]]

,channel_name,key_terms
0,Bolsas hoy | Invierte y Crece,"[S&P 500, Nasdaq, DAX, Eurostoxx, Oro, Cr..."
1,Bolsas hoy | Invierte y Crece,"[S&P 500 (SPX), ETF SPY, Nasdaq, ETF Triple..."
2,Bolsas hoy | Invierte y Crece,"[S&P 500, Nasdaq, Bitcoin, Oro, Acciones t..."
3,Bolsas hoy | Invierte y Crece,"[Inditex, Alphabet (Google), Walt Disney, Acci..."
4,Bolsas hoy | Invierte y Crece,"[IBEX, DAX, Eurostoxx, S&P 500, Nasdaq, Bitcoi..."
...,...,...
2220,Bolsas hoy | Esteban Pérez,"[Inditex, AXA, Mapfre, Bering, Sirius, Pfizer ..."
2221,Bolsas hoy | Esteban Pérez,"[Charter Communications (CHTR), Lemonade (LMN..."
2222,Bolsas hoy | Esteban Pérez,[Charter Communications (CHTR)]
2223,Bolsas hoy | Esteban Pérez,"[S&P 500, Nasdaq, Oro]"


In [ ]:
login(token=os.environ.get('HF_TOKEN'))

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [93]:
# Tomar el 10% de cada canal para test y el 90% para train
test_df = all_summaries_df.groupby("channel_name", group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))
train_df = all_summaries_df.drop(test_df.index)

print(f"Train shape: {train_df.shape}, Test shape: {test_df.shape}")

Train shape: (2004, 14), Test shape: (221, 14)


/var/folders/0z/3s22b27525jc3bm0m923h36h3wg489/T/ipykernel_8883/2328981104.py:2: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  test_df = all_summaries_df.groupby("channel_name", group_keys=False).apply(lambda x: x.sample(frac=0.1, random_state=42))


In [95]:
hf_train = Dataset.from_pandas(train_df)
hf_test = Dataset.from_pandas(test_df)

hf_dataset = DatasetDict({
    "train": hf_train,
    "test": hf_test
})

hf_dataset.push_to_hub("AndresR2909/youtube_transcriptions_summaries_2025_gpt4o")

Uploading the dataset shards: 100%|██████████| 1/1 [00:05<00:00,  5.45s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/AndresR2909/youtube_transcriptions_summaries_2025_gpt4o/commit/07ff353bf201ee1a8169f1484a49038740876b8a', commit_message='Upload dataset', commit_description='', oid='07ff353bf201ee1a8169f1484a49038740876b8a', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/AndresR2909/youtube_transcriptions_summaries_2025_gpt4o', endpoint='https://huggingface.co', repo_type='dataset', repo_id='AndresR2909/youtube_transcriptions_summaries_2025_gpt4o'), pr_revision=None, pr_num=None)